## Option pricing with the Heston model

The Heston Model models the volatility of the asset as a stochastic process. The price of the asset is given by

\begin{align}
dS_t & = \mu S_t dt + \sqrt{V_t} S_t dW_t^S \\[0.5ex]
dV_t &= \alpha(b-V_t)dt + \sigma \sqrt{V_t} dW_t^V 
\end{align}

and the correlation between the two Brownian motion is given by

\begin{equation}
dW_t^S dW_t^V = \rho \, dt.
\end{equation}

Using Feynman-Kac formula we can calculate the option price as an expectation value
\begin{align}
c_t & = \mathbf{E}[e^{-r(T-t)} {\rm max}(0,S_T-K)  \, \,  |  \, \, S_t] \\[0.5ex]
p_t & = \mathbf{E}[e^{-r(T-t)} {\rm max}(0,K-S_T)  \, \, |  \, \,S_t ]
\end{align}
where $c_t$ ($p_t$) corresponds to a call (put) option, $S_T$ is the stock price at maturity and $K$ is the strike price.



In [7]:
# ========================== Option price ===========================
# Author: Alexis D. Plascencia
# We implement the stochastic Heston Model for stock pricing
# We calculate the price of a Call and a Put option
# =====================================================================

import numpy as np
import matplotlib.pylab as plt
import scipy.stats as ss
import pandas as pd

from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import pyplot
from random import randint

from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

In [3]:
def Heston(paths, alpha, b, rho, mu, sigma, v0, s0):
    
    """Simulates Stock prices using the Heston Model

    Arguments:
        paths: The number of paths that will be generated
        alpha: Speed of reversion
        b: Long-term variance mean level
        rho: Correlation betwen Brownian motions
        mu: Drift of stock price
        sigma: Standard deviation of the standard deviation
        v0: Initial condition for volatility
        s0: Initial condition for the stock price

    Returns:
        vol: the volatility at each point in time.
        S: the stock price at each point in time.
    """

    # We generate the two corelated Brownian motions 
    means = np.array([0, 0])
    covs = np.matrix([[1, rho], [rho, 1]])
    W = ss.multivariate_normal.rvs( mean=means, cov=covs, size=(paths,steps) )

    W_S = W[:,:,0]   # Stock Brownian motion    
    W_V = W[:,:,1]   # Variance Brownian motion

    # We initialize the vectors
    vol = np.zeros((paths,steps+1))
    S = np.zeros((paths,steps+1))
    
    # We set the initial conditions
    vol[:,0] = v0
    S[:,0] = s0
    
    for i in range(steps):
        vol[:,i+1] = vol[:,i] + alpha*(b-vol[:,i])*dt + sigma*np.sqrt(vol[:,i])*np.sqrt(dt)*W_V[:,i]
        S[:,i+1] = S[:,i] + mu*dt*S[:,i] + np.sqrt(vol[:,i])*np.sqrt(dt)*W_S[:,i]*S[:,i] 
    
    # We return the volatility and the stock price as a function of time
    return vol, S

In [4]:
# Input Parameters:
steps = 1000                        # Number of small sub-steps (time)
ti = 0.0                            # Initial time (current time)
tf = 1.0                            # Maturity time
dt = (tf-ti)/steps                  # Time step
time = np.linspace(ti, tf, steps+1)

paths = 30000                       # Number of Monte Carlo paths      
alpha = 2.                          # Speed of reversion                           
b = 0.01                            # Long-term variance mean level                             
rho = 0.                            # Correlation betwen Brownian motions                              
mu = 0.0                            # Drift of stock price                         
sigma = 0.1                         # Standard deviation of the standard deviation                         
v0 = 0.01                           # Initial volatility                         
s0 = 105.0                          # Initial stock price                      
K = 100.0                           # Strike price

if 2*alpha*b>sigma**2.:
    print("Feller condition is satisfied")
else:
    print("Feller condition is NOT satisfied")

Feller condition is satisfied


In [9]:
# We generate different examples with the same input parameters
vol, S = Heston(paths, alpha, b, rho, mu, sigma, v0, s0)    

# Use the Feynman-Kac theorem to calculate the Call option price as an expectation value
call_price = np.mean(np.maximum(S[:,-1] - K, 0))
put_price = np.mean(np.maximum(K - S[:,-1] , 0))

print('The price for a Call option is = ', round(call_price,2))
print('The price for a Put option is  = ', round(put_price,2))

The price for a Call option is =  7.07
The price for a Put option is  =  2.03
